In [3]:
!pip install pymupdf

In [7]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define Drive path
DRIVE_FOLDER = "/content/drive/My Drive/AIData"
PDF_FOLDER = os.path.join(DRIVE_FOLDER, "PDFs")

# Create directories if they don't exist
os.makedirs(DRIVE_FOLDER, exist_ok=True)
os.makedirs(PDF_FOLDER, exist_ok=True)

print(f"All downloads will be saved in: {PDF_FOLDER}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
All downloads will be saved in: /content/drive/My Drive/BlockchainData/PDFs


In [9]:
import requests
import time
import os
import csv
import fitz


BASE_URL = "https://api.regulations.gov/v4"

API_KEY = "4MMLaxc6RObTFzeEoCyJt7VOGH7SzX5Qe8D9WzFd"

# Headers for API requests
HEADERS = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

# Keywords to Search
SEARCH_KEYWORDS = ["algorithmic transparency", "bias mitigation", "autonomous systems", "machine learning"]

# Folder to Save PDFs
SAVE_DIR = PDF_FOLDER
os.makedirs(SAVE_DIR, exist_ok=True)

#CSV File to Store Extracted Data ---> this is under review
CSV_FILE = os.path.join(DRIVE_FOLDER, "regulations_data.csv")

#Handle Rate Limits
def handle_rate_limits(response):
    if response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After", 10))
        print(f"Rate limit hit! Sleeping for {retry_after} seconds...")
        time.sleep(retry_after)
        return True
    return False

#Retrieve Documents Based on Keywords
def get_documents():
    documents = []
    page = 1

    while len(documents) < 1000:  # Stop after collecting 1000 documents
        params = {
            "filter[searchTerm]": " OR ".join(SEARCH_KEYWORDS),  # Search for multiple keywords
            "page[size]": 100,  # Fetch 100 docs per page
            "page[number]": page
        }

        response = requests.get(f"{BASE_URL}/documents", headers=HEADERS, params=params)

        if handle_rate_limits(response):
            continue

        if response.status_code != 200:
            print(f"Failed to retrieve page {page}: {response.status_code}")
            print("Error details:", response.text)
            break

        data = response.json()
        items = data.get("data", [])

        if not items:
            print("No more documents found.")
            break

        documents.extend(items)
        print(f"Retrieved {len(items)} documents from page {page}")

        page += 1  # Move to next page

    return documents[:1000]  # Return only 1000 documents

#Extract Attachments and Download PDFs
def get_attachments(doc_id):
    attach_url = f"{BASE_URL}/documents/{doc_id}/attachments"
    response = requests.get(attach_url, headers=HEADERS)

    if handle_rate_limits(response):
        return None

    if response.status_code != 200:
        print(f"Failed to fetch attachments for {doc_id}")
        return None

    attachments = response.json().get("data", [])
    for attachment in attachments:
        file_formats = attachment.get("attributes", {}).get("fileFormats", [])
        if file_formats:
            return file_formats[0]["fileUrl"]  # Return first available file URL

    return None  # No attachments found

#Download PDFs and Extract Text
def download_pdf(doc_id, file_url):
    file_path = os.path.join(SAVE_DIR, f"{doc_id}.pdf")

    response = requests.get(file_url)
    if handle_rate_limits(response):
        return None

    if response.status_code == 200:
        with open(file_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded PDF: {file_path}")
        return file_path
    else:
        print(f"Failed to download PDF for {doc_id}")
        return None

#Step 4: Extract Text from PDFs
def extract_text_from_pdf(pdf_path):
    try:
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text("text") + "\n"
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return None

#Step 5: Save Data to CSV
def save_to_csv(documents):
    with open(CSV_FILE, "w", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["document_id", "title", "docket_id", "abstract", "document_text"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for doc in documents:
            doc_id = doc.get("id")
            title = doc.get("attributes", {}).get("title", "Unknown")
            docket_id = doc.get("attributes", {}).get("docketId", "Unknown")
            abstract = doc.get("attributes", {}).get("abstract", "No abstract available")

            # Fetch attachments & download PDFs
            file_url = get_attachments(doc_id)
            if file_url:
                pdf_path = download_pdf(doc_id, file_url)
                if pdf_path:
                    document_text = extract_text_from_pdf(pdf_path)
                else:
                    document_text = "Failed to download PDF"
            else:
                document_text = "No PDF available"

            writer.writerow({
                "document_id": doc_id,
                "title": title,
                "docket_id": docket_id,
                "abstract": abstract,
                "document_text": document_text
            })

            print(f"Saved document {doc_id} to CSV")


documents = get_documents()
if documents:
    save_to_csv(documents)


Retrieved 100 documents from page 1
Retrieved 100 documents from page 2
Retrieved 100 documents from page 3
Retrieved 100 documents from page 4
Retrieved 100 documents from page 5
Retrieved 100 documents from page 6
Retrieved 100 documents from page 7
Retrieved 100 documents from page 8
Retrieved 40 documents from page 9
No more documents found.
Downloaded PDF: /content/drive/My Drive/BlockchainData/PDFs/OMB-2023-0020-0198.pdf
Saved document OMB-2023-0020-0198 to CSV
Downloaded PDF: /content/drive/My Drive/BlockchainData/PDFs/EPA-HQ-OAR-2015-0072-1606.pdf
Saved document EPA-HQ-OAR-2015-0072-1606 to CSV
Downloaded PDF: /content/drive/My Drive/BlockchainData/PDFs/EPA-HQ-OAR-2015-0072-1609.pdf
Saved document EPA-HQ-OAR-2015-0072-1609 to CSV
Saved document NRC-2022-0095-0011 to CSV
Downloaded PDF: /content/drive/My Drive/BlockchainData/PDFs/EPA-HQ-OAR-2015-0072-1638.pdf
Error extracting text from /content/drive/My Drive/BlockchainData/PDFs/EPA-HQ-OAR-2015-0072-1638.pdf: Failed to open file